Got inspired [here](https://developers.google.com/earth-engine/tutorials/community/time-series-visualization-with-altair)

In [ ]:
import ee
import geemap
from geemap import ee_initialize
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

ee_initialize()

In [ ]:

from create_reduce_region_function import create_reduce_region_function
from fc_to_dict import fc_to_dict
from add_date_info import add_date_info
from addNDVI import addNDVI
from do_buffer import do_buffer

In [ ]:
stations = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point(10.5798451965, 46.6862577024), {'name': 'B2 (1490m)'}),
  ee.Feature(ee.Geometry.Point(10.5918853513, 46.6917328627), {'name': 'B3 (1930m)'}),
  ee.Feature(ee.Geometry.Point(10.5850561988, 46.684249288), {'name': 'P2 (1549m)'})])

aoi = stations.map(do_buffer)
aoi_all = aoi.geometry()
aoi_sel = ee.Geometry.Point(10.5850561988, 46.684249288)

In [ ]:
s2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2016-01-01', '2019-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .filterBounds(aoi_all)

In [ ]:
ndvi = s2.map(addNDVI).select("NDVI")
ndvi_bands = ndvi.toBands().clip(aoi_all)

In [ ]:
ndvi_vis = {
  'min': -1.0,'max': 1,
  'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', 
  '74A901','66A000', '529400', '3E8601', '207401', '056201', '004C00', 
  '023B01','012E01', '011D01', '011301']
}

Map = geemap.Map(center=[ 46.688,10.586], zoom = 15)
Map.add_basemap("SATELLITE")
Map.addLayer(ndvi_bands,name="NDVI_series")
Map.addLayer(ndvi_bands.select(4),ndvi_vis,name="NDVI_image")
Map.addLayer(stations,name="Monitoring Stations")
Map.set_plot_options(plot_type='bar',add_marker_cluster=False, marker=None)
Map.roi_reducer = ee.Reducer.mean()
Map

In [ ]:

reduce_ndvi = create_reduce_region_function(
    geometry=aoi_sel, reducer=ee.Reducer.median(), scale=10)

ndvi_stat_fc = ee.FeatureCollection(ndvi.map(reduce_ndvi)).filter(
    ee.Filter.notNull(ndvi.first().bandNames()))

In [ ]:
ndvi_dict = fc_to_dict(ndvi_stat_fc).getInfo()
ndvi_df = pd.DataFrame(ndvi_dict)

In [ ]:
ndvi_df = add_date_info(ndvi_df).drop(['millis','system:index'],axis=1)
ndvi_df.head(5)

In [ ]:
fig = px.line(ndvi_df, x="DOY", y="NDVI", color='Year')
f = go.FigureWidget(fig)
f